In [1]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_excel("stock_data.xlsx")

# Vorverarbeitung

In [6]:
df2 = df.sort_values(by=['Unternehmen', 'Datum'])[['Datum', 'Schluss', 'Unternehmen']]
df2.head()

,Datum,Schluss,Unternehmen
727,2020-02-17,23.32,1&1 Drillisch
726,2020-02-18,22.92,1&1 Drillisch
725,2020-02-19,23.02,1&1 Drillisch
724,2020-02-20,23.42,1&1 Drillisch
723,2020-02-21,23.94,1&1 Drillisch


## Veränderung zum Vortag

In [25]:
df2['SchlussVortag'] = df2.Schluss.shift()
df2['UnternehmenVortag'] = df2.Unternehmen.shift()

In [26]:
df2['change'] = df2.Schluss / df2.SchlussVortag

In [33]:
df2.loc[df2.Unternehmen!=df2.UnternehmenVortag, 'change'] = None
df2.loc[df2.Unternehmen!=df2.UnternehmenVortag, 'SchlussVortag'] = None

In [35]:
df2.iloc[45:55]

,Datum,Schluss,Unternehmen,SchlussVortag,change,UnternehmenVortag
682,2020-04-22,19.47,1&1 Drillisch,19.29,1.009331,1&1 Drillisch
681,2020-04-23,20.13,1&1 Drillisch,19.47,1.033898,1&1 Drillisch
680,2020-04-24,19.26,1&1 Drillisch,20.13,0.956781,1&1 Drillisch
679,2020-04-27,19.78,1&1 Drillisch,19.26,1.026999,1&1 Drillisch
678,2020-04-28,20.11,1&1 Drillisch,19.78,1.016684,1&1 Drillisch
677,2020-04-29,21.00,1&1 Drillisch,20.11,1.044257,1&1 Drillisch
676,2020-04-30,21.24,1&1 Drillisch,21.00,1.011429,1&1 Drillisch
2339,2020-02-17,12.48,ADLER Real Estate,NaN,NaN,1&1 Drillisch
2338,2020-02-18,12.58,ADLER Real Estate,12.48,1.008013,ADLER Real Estate
2337,2020-02-19,12.84,ADLER Real Estate,12.58,1.020668,ADLER Real Estate


In [37]:
px.line(df2, x='Datum', y='change', color='Unternehmen')

# Resampling

In [38]:
df_ws = pd.read_parquet("transaction_history.parquet")

In [42]:
df_ws[df_ws.basket_id=='90c596d8-d47b-425a-b64e-7ccc3954ac6a']

,date,weekday,region,state,age,age_group,basket_id,product,product_name,product_group,sales_value
0,2020-01-01,Wednesday,North,de-sh,23,18-29,90c596d8-d47b-425a-b64e-7ccc3954ac6a,1,red wine,1,4.3
1,2020-01-01,Wednesday,North,de-sh,23,18-29,90c596d8-d47b-425a-b64e-7ccc3954ac6a,2,white wine,1,3.8
2,2020-01-01,Wednesday,North,de-sh,23,18-29,90c596d8-d47b-425a-b64e-7ccc3954ac6a,4,pils,2,53.8
3,2020-01-01,Wednesday,North,de-sh,23,18-29,90c596d8-d47b-425a-b64e-7ccc3954ac6a,5,altbier,2,7.9
4,2020-01-01,Wednesday,North,de-sh,23,18-29,90c596d8-d47b-425a-b64e-7ccc3954ac6a,6,lager,2,31.2


In [47]:
df_viz = df_ws.groupby(['date', 'product_name'], as_index=False)[['sales_value']].sum()
df_viz

,date,product_name,sales_value
0,2020-01-01,altbier,768.9
1,2020-01-01,grappa,588.5
2,2020-01-01,lager,2018.5
3,2020-01-01,pils,2345.6
4,2020-01-01,red wine,1541.0
...,...,...,...
1484,2020-06-14,red wine,3617.0
1485,2020-06-14,rosé wine,1033.7
1486,2020-06-14,vodka,631.9
1487,2020-06-14,whiskey,1661.8


In [49]:
px.line(df_viz, x='date', y='sales_value', color='product_name', markers=True)

In [50]:
df_viz = df_ws.groupby('date', as_index=False)[['sales_value']].sum()
px.line(df_viz, x='date', y='sales_value', markers=True)

In [56]:
df_wsi = df_ws.set_index('date')
df_ws.groupby('date').agg({'sales_value':'sum'}).head(3)

,sales_value
date,
2020-01-01,11755.5
2020-01-02,11570.2
2020-01-03,14193.0


In [70]:
df_viz = df_wsi.resample('7D').agg({'sales_value':'sum'})
px.line(df_viz.reset_index(), x='date', y='sales_value')